> #### 1. 그래프로 표현될 수 있는 데이터
> #### 2. 그래프 데이터에서의 예측 문제 종류, 그래프 표현법
> #### 3. GNN
> #### 4. GNN의 실제 데이터 적용
> #### 5. GNN 관련 다양한 주제들

그래프: Vertex(또는 Node)와 Edge(또는 Link)로 구성되어, 개체간의 관계를 나타내는 자료구조

요소별 속성(Attribute)
- Vertex 속성: Vertex가 나타내는 것, 이웃의 수 등
- Edge 속성: Edge가 나타내는 것, Edge weight 등
- Global 속성: Vertex의 수, 가장 긴 경로 등

각 속성을 하나의 Tensor로 묶어 놓은 것을 **Embedding**이라고 한다. 🌐

Edge가 방향성을 가지는지 여부에 따라 Directed, Undirected 그래프로 나뉜다.

## 1. 그래프로 표현될 수 있는 데이터

#### Image as graph
각 픽셀을 Node로, 픽셀 간에 이웃해 있는 것을 Edge로 생각할 수 있다. 🌐

#### Texts as graph
문장의 각 글자, 단어 등을 Node로, 글자나 단어 간에 이웃해 있는 것을 Edge로 생각할 수 있다. 🌐

위의 두 경우, 그래프로 표현된 결과가 매우 규칙적인 구조를 가지고, 매우 Sparse하므로 공간효율적인 표현법을 고려할 수 있다.

#### Molecules as graphs
원자와 전자로 이루어지는 분자. 원자를 Node로, 원자간의 공유결합을 Edge로 생각하여, 분자의 3차원 구조를 그래프로 추상화 할 수 있다. 🌐

#### Social networks as graphs
인물, 조직을 Node로, 그들간의 상호작용을 Edge로 생각할 수 있다. 🌐

#### Citation networks as graphs
논문을 Node로, 논문 간의 인용을 Edge로 생각할 수 있다. 

#### Other
ML 코드, 프로그래밍 코드, 수학 방정식 등 객체 간 관계 시각화.

### Example: Zachary's Karate Club
Karate Club 총 34명의 멤버(Node)가 총 78개의 상호작용(Edge)을 가지고 각자가 4개의 그룹 중 하나에 속해 있음.

![zachary_circle](imgs/zachary_circle.png)

## 2. 그래프 데이터에서의 예측 문제 종류, 그래프 표현법

### 그래프 데이터가 주어졌을 때 어떤 종류의 예측을 수행할 수 있을까?

1. 그래프 단위 태스크

> 전체 그래프의 성질을 예측하는 것<br>
> 그래프가 분자 구조를 표현한다고 했을 때, 분자가 어떤 냄새를 낼지, 질병에 있어서 반응체에 결합할지 등을 예측

2. Node 단위 태스크

> 어떤 Node의 그래프에서의 역할 등 예측<br>
> Zachary's Karate Club 예제. 클럽 내 두 파벌이 생기고, 각 멤버들이 어떤 파벌에 속하는지 예측하는 문제. 🌐

3. Edge 단위 태스크

> 그래프 상에서 두 Node간에 Edge가 존재하는지, 존재한다면 어떤 관계성을 의미하는 지<br>
> ![edge_level_task](imgs/edge_level_task.png)

### ML에 적절한 그래프 표현법

Node, Edge, 그래프의 속성은 간단하게 행렬 형태로 표현 가능  
(각 요소의 개수 x 해당 요소가 가지는 feature의 갯수)

Graph connectivity의 경우 까다로움  
인접 행렬의 문제점
> 1. Sparse 그래프의 경우, 매우 공간비효율적
> 2. 동일한 그래프에 대해서 Node의 배치 순서에 따라 다른 행렬로 표현 가능 &rarr; **학습의 결과가 달라짐!** (not permutation invariant) 🌐

인접 리스트 : 출발 Node와 도착 Node로 이루어진 Tuple을 저장. Sparse 그래프의 저장에 효율적. **permutation invariant** &rarr; ML input으로 사용하기에 적합. 🌐

## 3. GNN
**Graph connectivity를 그대로 유지**하면서 Vertex, Edge, 그래프 속성 **(Embedding)을 Update**

#### Simplest GNN
![simplest_gnn](imgs/simplest_gnn.png)

![final_prediction](imgs/final_prediction.png)
최종 Embedding에 Linear classifier를 적용해서 Node 분류 문제 해결 가능

#### Pooling
예측하고자 하는 요소에 대한 정보가 없는 경우는?  
소셜 네트워크 분석시에, 익명성 유지를 위해 유저 데이터를 사용할 수 없다면?  
다른 요소의 정보를 함께 고려 &rarr; **Pooling** 🌐
> 1. 합칠 요소들의 최종 Embedding을 겹침
> 2. 여러 Embedding을 통합함(보통 총합을 내는 식으로)

Edge &rarr; Node Pooling

![edge_to_node_pooling](imgs/edge_to_node_pooling.png)

Node &rarr; Edge Pooling

![node_to_edge_pooling](imgs/node_to_edge_pooling.png)

Node &rarr; 그래프 Pooling
![node_to_global_pooling](imgs/node_to_global_pooling.png)

#### Graph Convolutional Network (GCN)

Pooling은 최종 Embedding에서 여러 요소의 정보들을 종합하는 것.

GNN 레이어 내에서는?

**Message Passing**: 주변 Node 혹은 Edge 들과의 정보 교환을 통해 Embedding을 Update 🌐
> 1. Node A의 이웃 Node들의 Embedding(**Message**)을 겹침
> 2. 겹쳐진 Embedding을 통합함(보통 총합을 내는 식으로)
> 3. 통합된 Embedding에 update function을 적용하여 Node A의 Embedding을 업데이트

![gcn](imgs/gcn.png)

한 레이어를 통과할 때마다 이웃 Node, 이웃의 이웃 Node, 이웃의 이웃의 이웃 Node, ...의 Message가 전달됨  
왜 **이웃**의 정보를 종합할까?  
**유사한 노드에 연결되어 있을 가능성이 그렇지 않은 노드에 연결될 가능성보다 높다**는 전제  
**이웃의 정보를 받음으로써 경향성을 강화** &rarr; 예측력 강화



#### Image convolution과 유사성

![image_convolution](imgs/image_convolution.png)

**인접한** 픽셀의 feature vector를 모두 더해서 구한 평균값으로 픽셀의 feature vector를 업데이트

### Example: Zachary's Karate Club

멤버 개개인에 관한 정보(Node attribute)는 주어지지 않고, 멤버 간 상호작용 여부 (Graph connectivity)만 주어짐.

목표: 그래프가 주어졌을 때, 각 멤버가 속하는 그룹을 분류하는 모델

![initial_karate](imgs/initial_karate.png)

![final_karate](imgs/final_karate.png)

단일 레이어로 이루어진 GNN 모델을 구현

$X→H(X)→ReLU(H(X))→Z(ReLU(H(X)))$  
$X$: Input Node 속성 행렬(이 데이터의 경우 Identity 행렬)  
$H$: GCN 레이어 / $H(X)$: Update된 Embedding  
$Relu$: Activation 함수  
$Z$: Linear classifier

위의 모델로 학습 진행. Training 데이터셋을 대상으로 성능 평가함. 🌐  
Epoch가 늘어나면서 구성 함수의 Parameter가 데이터셋에 맞게 조정되고, 정확도가 증가.

Epoch가 늘어나면서 Embedding($H(X)$)이 분류에 적합하게 Update됨 🌐